In [1]:
import sys

sys.path.append('../../')

from library.monday_api import MondayApi, MondayBoard
from library.consts import API_URL, PRODUCT_BOARD_ID, ORDERS_BOARD_ID , DONATIONS_BOARD_ID
import uuid
import json
import pandas as pd
import os

api_key = os.environ['RESTART_MONDAY_API_KEY']

orders_board_id = 1296415271 #test
# orders_board_id = ORDERS_BOARD_ID

def create_order(api_key, dto):
    monday_api = MondayApi(api_key, API_URL)
    monday_board = MondayBoard(monday_api, id=orders_board_id)

    order_id = uuid.uuid4().hex

    new_item = monday_board.insert_item(dto['name'], {
        'long_text': dto['note'],  # Note
        'text8': dto['phone'],  # Phone
        'text0': dto['unit'],  # Unit
        'text7': dto['job'],  # Job
        'email': {"email": dto['email'], "text": ""},  # Email
        'dropdown': dto['location'],  # Location
        'color': {"label": dto['tenant']},  # Tenant Code
        'text5': order_id,  # Custom Id
        'status4': {"label": dto.get('type', 'IDF')},  # Form Type
    })
    new_item_id = int(new_item['create_item']['id'])

    for subitem in dto.get('subitems', []):
        monday_board.insert_subitem(subitem['name'], {
            'connect_boards': {"linkedPulseIds": [{"linkedPulseId": subitem['product_number']}]},  # Product
            'numbers': subitem['quantity'],  # Quantity
        }, new_item_id)

    return order_id


def get_products(api_key):
    monday_api = MondayApi(api_key, API_URL)
    monday_board = MondayBoard(monday_api, id=PRODUCT_BOARD_ID)
    items = monday_board.get_items(return_items_as='json').get('data').get('boards')[0].get('items_page').get('items')

    products = [{
        'name': i.get('name'),
        'product_number': int(i.get('id')),
        'type': next((z['text'] for z in i.get('column_values') if z['id'] == 'dropdown'), None)
    } for i in items]

    return products



In [2]:
get_products(api_key)

[{'name': 'חליפת סערה', 'product_number': 1289515525, 'type': 'IDF'},
 {'name': 'נעלי הרים', 'product_number': 1289515528, 'type': 'IDF'},
 {'name': 'פנסים לנשק', 'product_number': 1289516493, 'type': 'IDF'},
 {'name': 'כוונת M16', 'product_number': 1289516504, 'type': 'IDF'},
 {'name': 'חסם עורקים', 'product_number': 1289516557, 'type': 'IDF'},
 {'name': 'שלוקרים', 'product_number': 1289516571, 'type': 'IDF, EMR'},
 {'name': 'מדים טקטיים', 'product_number': 1289516863, 'type': 'IDF, EMR'},
 {'name': 'תחבושות אישיות', 'product_number': 1289516873, 'type': 'IDF'},
 {'name': 'תיקי שורש', 'product_number': 1289516877, 'type': 'IDF'},
 {'name': 'קסדות טקטיות', 'product_number': 1289516882, 'type': 'IDF'},
 {'name': 'וסטים קרמים', 'product_number': 1289516887, 'type': 'IDF'},
 {'name': 'פלטות קרמיות', 'product_number': 1289516892, 'type': 'IDF'},
 {'name': 'ברכיות', 'product_number': 1289516898, 'type': 'IDF'},
 {'name': 'פנס ראש', 'product_number': 1290168897, 'type': 'IDF, EMR'},
 {'name'

In [5]:

# Read the Excel file into a DataFrame
df = pd.read_excel('first_ex.xlsx')

# Define a function to format each row into the desired structure
def format_row(row):
    formatted_row = {
        'name': row['שם איש קשר'],
        'note': f"{row['הערות']}\n{row['מיקום בארץ של החיילים']}\n{row['מה צריך']}",
        'phone': row['טלפון איש קשר'], # Need to change
        'unit': row['שם ממבקש'],
        'job': 'לא ידוע',
        'email': "",  # You can set a default value or leave it empty
        'location': 'אחר',
        'tenant': 'dora',
        # 'status': row['סטאטוס'],
        'status': 'ממתין',
        'subitems': []
    }
    
    items_dict = {
        'אפודי מגן': 1289516887,
        'ידיות סער': 1297560425,
        'קסדות ': 1289516882,
        'שלוקרים': 1289516571,
        'ווסטים קרמיים': 1289516887,
        'מחסניות': 1296405777
    }
    
    for key in items_dict.keys():
        value = row[key]
        if value and pd.notna(value):
            try:
                quantity = int(value)
            except ValueError:
                # Handle non-numeric values (e.g., set to 0 or a default value)
                quantity = 0 # it means they wrote something
            formatted_row['subitems'].append({
                'name': key,
                'product_number': items_dict[key],
                'quantity': quantity        
            })
    
    return formatted_row

# Apply the format_row function to each row in the DataFrame
formatted_data = df.apply(format_row, axis=1)

# Convert the resulting DataFrame to a list of dictionaries
formatted_data_list = formatted_data.to_list()

for item in formatted_data_list:
    # create_order(api_key, item)
    print(item)


Exception: ERROR! - request status (401)